# Lab 2 - Repeatable Patterns

> **⚠️ Browser Compatibility Notice**: This lab requires **Google Chrome browser** for optimal performance. Please ensure you're using Chrome before proceeding.

> **Important**: If you haven't completed Lab 1, please ensure you've followed the Code Setup Instructions in Lab 1 before proceeding.

In this section, we'll explore common reusable patterns for integrating Nova Sonic voice chat, including:

- Integrate with MCP
- Integrate with Strands Agents

## Design Trade-offs in Sonic Agentic Workflow Integration

The **MCP lab** demonstrates a pattern that leverages Nova Sonic's built-in reasoning model to manage tool selection and directly invoke external MCP tools. This approach is well-suited for use cases requiring relatively simple toolUse definitions. Its key advantage is reduced latency, as all reasoning and generation happen within Sonic.

The **Strands Agent samples** illustrate a different pattern: keeping the toolUse definition within Sonic simple while delegating reasoning and generation to an external agentic workflow. This approach offers greater flexibility and allows users to reuse existing workflows with minimal refactoring. However, it typically introduces higher latency due to redundant steps in tool selection, reasoning, and generation.

## Integrate with MCP

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro) is an open standard that enables AI models to access external data and tools. This sample integrates with MCP to support location-based queries using the [AWS Location Service MCP server](https://github.com/awslabs/mcp?tab=readme-ov-file#aws-location-service-mcp-server).

### Tool Routing by Nova Sonic

![Sonic MCP Processing](static/image-36.png)

The workflow:
1. Users ask questions via voice chat, such as: "Where is the largest zoo in Seattle?"
2. Nova Sonic triggers a toolUse event with the specified ToolName (e.g., `search_places`, `search_nearby`)
3. Nova Sonic calls the MCP server tool directly and retrieves the raw response
4. Nova Sonic processes the Tool response and generates output

### Tool Configuration

Located at: [`react-client/src/helper/s2sEvents.js`](react-client/src/helper/s2sEvents.js#L28-L67) - see the `DEFAULT_TOOL_CONFIG` starting at line 28

```javascript
{
  "toolSpec": {
    "name": "locationMcpTool",
    "description": "Access location services to find places, addresses, nearby locations.",
    "inputSchema": {
      "json": JSON.stringify({
        "type": "object",
        "properties": {
          "tool": {
            "type": "string",
            "description": "The function name to search the location service. One of: search_places, get_place, search_nearby, reverse_geocode",
          },
          "query": {
            "type": "string",
            "description": "The search query to find relevant information"
          }
        },
        "required": ["query"]
      })
    }
  }
}
```

### Handle MCP ToolUse Event

In [`python-server/s2s_session_manager.py`](python-server/s2s_session_manager.py#L295-L298) - see the MCP integration logic around line 295:

```python
# MCP integration - location search                        
if toolName == "getlocationtool":
    if self.mcp_loc_client:
        result = await self.mcp_loc_client.call_tool(content)
```

The MCP client in [`python-server/integration/mcp_client.py`](python-server/integration/mcp_client.py#L50-L60) calls the tool directly to minimize latency - see the `call_tool` function starting at line 50:

```python
async def call_tool(self, input):
    if isinstance(input, str):
        input = json.loads(input)
    
    tool_name = input.get("tool", "search_places")
    query = input.get("query", input)
    
    response = await self.session.call_tool(tool_name, {"query": query})
    result = []
    for c in response.content:
        result.append(c.text)
    return result
```

### Lab: Test MCP Integration

1. Restart the Python WebSocket server to enable MCP integration:
   - Press `Ctrl + C` to stop the service
   - Run with MCP enabled:
     ```bash
     python server.py --agent mcp
     ```

2. Choose **MCP - get location** from the test profile dropdown:

   ![Test Profile MCP](static/image-37.png)

3. Try questions like:
   - "Find me the location of the largest zoo in Seattle, Washington."
   - "Find the largest shopping mall in New York City."

## Integrate with Strands Agents

[Strands Agents](https://strandsagents.com/0.1.x) is an open source SDK that takes a model-driven approach to building and running AI agents. In this lab, we demonstrate an agentic workflow pattern using Strands Agent to delegate complex reasoning and orchestration to an external integration.

### Tool Routing by External Agentic Workflow

![Sonic Strands Processing](static/image-38.png)

The Strands Agents use two tools:
- AWS Location Service MCP server
- Weather API wrapped as a Strands Tool

The Weather tool requires latitude and longitude, so the agent first gets geo-coordinates from the location service before retrieving weather data. This is handled automatically by Strands Agents.

The workflow:
1. Users ask: "What's the weather like in Seattle today?"
2. Nova Sonic uses a generic ToolUse definition and returns the ToolName
3. Nova Sonic calls the Strands Agents with the input 'weather in Seattle'
4. Strands Agents apply reasoning to determine orchestration, call tools, and generate response
5. Nova Sonic processes the Agent response and generates audio output

> **Note**: Strands Agents use Nova Lite for reasoning (lower latency). For complex logic, you may opt for a larger model with increased latency.

### Tool Configuration

Located at: [`react-client/src/helper/s2sEvents.js`](react-client/src/helper/s2sEvents.js#L28-L67) - see the `DEFAULT_TOOL_CONFIG` starting at line 28

```javascript
{
  "toolSpec": {
    "name": "externalAgent",
    "description": "Get weather information for specific locations.",
    "inputSchema": {
      "json": JSON.stringify({
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The search query to find relevant information"
          }
        },
        "required": ["query"]
      })
    }
  }
}
```

### Handle ToolUse Event

In [`python-server/s2s_session_manager.py`](python-server/s2s_session_manager.py#L300-L303) - see the Strands Agent integration around line 300:

```python
# Strands Agent integration - weather questions
if toolName == "externalagent":
    if self.strands_agent:
        result = self.strands_agent.query(content)
```

The Strands Agents implementation in [`python-server/integration/strands_agent.py`](python-server/integration/strands_agent.py#L9-L25) - see the `weather` function starting at line 9 and the `StrandsAgent` class starting at line 27:

```python
@tool
def weather(lat, lon: float) -> str:
    """Get weather information for a given lat and lon"""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": str(lat),
        "longitude": str(lon),
        "current_weather": True
    }
    response = requests.get(url, params=params)
    return response.json()["current_weather"]

class StrandsAgent:
    def __init__(self):
        # Launch AWS Location Service MCP Server
        self.aws_location_srv_client = MCPClient(lambda: stdio_client(
            StdioServerParameters(
                command="uvx", 
                args=["awslabs.aws-location-mcp-server@latest"],
                env=env)
            ))
        
        # Create Strands Agent with Nova Lite
        bedrock_model = BedrockModel(
            model_id="amazon.nova-lite-v1:0",
            boto_session=session
        )
        tools = self.aws_location_srv_tools
        tools.append(weather)
        self.agent = Agent(
            tools=tools, 
            model=bedrock_model,
            system_prompt="You are a chat agent tasked with answering location and weather-related questions."
        )
```

### Lab: Test Strands Agents Integration

1. Restart the Python WebSocket server:
   - Press `Ctrl + C` to stop the service
   - Run with Strands enabled:
     ```bash
     python server.py --agent strands
     ```

2. Choose **Strands Agents - get weather** from the test profile dropdown:

   ![Test Profile Strands](static/image-39.png)

3. Ask: "What's the weather like in Seattle today?"

4. In the Python WebSocket terminal, you should see the Strands Agent's "thinking" process.

## Key Takeaways

In this lab, you've learned about:

✅ **MCP Integration**: Using Model Context Protocol for external tool access with minimal latency
✅ **Strands Agents**: Delegating complex reasoning to external agentic workflows
✅ **Design Trade-offs**: Understanding when to use different integration patterns based on complexity and latency requirements
✅ **Tool Configuration**: Setting up different types of tools for various use cases

## Architecture Patterns Summary

- **Direct Nova Sonic Tools**: Best for simple, low-latency interactions
- **MCP Integration**: Ideal for standardized external tool access
- **Strands Agents**: Perfect for complex reasoning and multi-step workflows
- **RAG with Knowledge Bases**: Essential for domain-specific information retrieval

## Additional Resources

- [Amazon Nova Sonic Documentation](https://docs.aws.amazon.com/nova/latest/userguide/speech.html)
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)
- [Strands Agents Documentation](https://strandsagents.com/0.1.x)
- [Amazon Nova Samples GitHub Repository](https://github.com/aws-samples/amazon-nova-samples)
- [AWS Location Service MCP Server](https://github.com/awslabs/mcp)